# Глава 30. Шаблоны проектирования с классами

## Python и ООП

Реализацию ООП в языке Python можно свести к трем следующим идеям:

* **Наследование** основано на механизме поиска атрибутов в языке Python (в выражении `X.name`)


* **Полиморфизм** Назначение метода `method` в выражении `X.method` зависит от типа (класса) `X`. Разрешение имен атрибутов производится на  этапе выполнения, объекты, реализующие одинаковые интерфейсы, являются взаимозаменяемыми - клиентам не требуется знать тип объекта, реализующего вызываемый метод.


* **Инкапсуляция** Методы и операторы реализуют поведение; сокрытие данных - это соглашение по умолчанию. Инкапсуляция - упаковывание, сокрытие подробностей реализации за интерфейсами объекта, она позволяет изменять реализацию интерфейсов объекта, не оказывая влияния на пользователей этого объекта.

### Перегрузка посредством сигнатур вызова (точнее, ее невозможность)

В некоторых ООЯ под полиморфизмом также понимается возможность перегрузки функций, основанной на сигнатурах типов их аргументов. Но так как в языке Python отсутствуют объявления типов, эта концепция в действительности здесь неприменима - **полиморфизм в языке Python основан на интерфейсах объектов, а не на типах**.

Следует писать такой код, который опирается на интерфейс объекта, а не на конкретный тип данных:

```
class C:
    def meth(self, x):
        x.operation()  # предпол-ся, что x работает правильно
```

## ООП и наследование: взаимосвязи типа «является»

С точки зрения проектировщика, наследование - это способ указать принадлежность к некоторому набору: класс определяет набор свойств, которые могут быть унаследованы и адаптированы более специализированными наборами (то есть подклассами).

Пример с пиццерией. Коллектив работников можно определить четырьмя классами.

Самый общий класс, `Employee`, реализует поведение, общее для всех работников, такое как повышение заработной платы (`giveRaise`) и вывод на экран (`__repr__`).

Существует две категории служащих и, соответственно, два подкласса, наследующих класс `Employee`: `Chef` (повар) и `Server` (официант). Оба подкласса переопределяют унаследованный метод `work`, чтобы обеспечить вывод более специализированных сообщений.

Наконец, робот по приготовлению пиццы моделируется еще более специализированным классом `PizzaRobot`, наследующий класс `Chef`.

В терминах ООП мы называем такие взаимоотношения **"является"**: робот является поваром, а повар является служащим.

In [1]:
# epmloyees.py

class Employee:
    def __init__(self, name, salary=0):
        self.name = name
        self.salary = salary
        
    def giveRaise(self, percent):
        self.salary = self.salary + (self.salary * percent)
        
    def work(self):
        print(self.name, 'does stuff')
        
    def __repr__(self):
        return '<Employee: name=%s, salary=%s>' % (self.name, self.salary)

    
class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 50000)
        
    def work(self):
        print(self.name, 'makes food')

        
class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 40000)

    def work(self):
        print(self.name, 'interfaces with customer')

        
class PizzaRobot(Chef):
    def __init__(self, name):
        Chef.__init__(self, name)

    def work(self):
        print(self.name, 'makes pizza')

        
if __name__ == '__main__':
    bob = PizzaRobot('bob') # Создать робота с именем bob
    print(bob)              # Вызвать унаследованный метод __repr__
    bob.work()              # Выполнить действие, зависящее от типа
    bob.giveRaise(0.20)     # Увеличить роботу зарплату на 20%
    print(bob); print()

    for klass in Employee, Chef, Server, PizzaRobot:
        obj = klass(klass.__name__)
        obj.work()

<Employee: name=bob, salary=50000>
bob makes pizza
<Employee: name=bob, salary=60000.0>

Employee does stuff
Chef makes food
Server interfaces with customer
PizzaRobot makes pizza


## ООП и композиция: взаимосвязи типа «имеет»

С точки зрения программиста, композиция - это прием встравивания других объектов в объект-контейнер и использование их для реализации методов контейнера.

Для проектировщика композиция - это один из способов представить взаимоотношения в прикладной области. Но вместо того, чтобы определять принадлежность к множеству, при композиционном подходе все части объединяются в единое целое.

Кроме того, **композиция отражает взаимоотношения между частями, которые обычно называются отношениями типа "имеет"**. Иногда композиция называется **агрегированием**.

В примере выше, объединим коллектив работников в коллектив пиццерии и позволим им приступить к работе. Пиццерия - составной объект: в нем имеется печь и работники, такие как официанты и повара. Когда приходит клиент и делает заказ, все компоненты пиццерии начинают действовать.

In [2]:
# pizzashop.py

# from employees import PizzaRobot, Server

class Customer:
    def __init__(self, name):
        self.name = name

    def order(self, server):
        print(self.name, 'orders from', server)

    def pay(self, server):
        print(self.name, 'pays for item to', server)

        
class Oven:
    def bake(self):
        print('oven bakes')

        
class PizzaShop:
    def __init__(self):
        self.server = Server('Pat')    # Встроить другие объекты
        self.chef = PizzaRobot('Bob')  # Робот по имени Bob
        self.oven = Oven()

    def order(self, name):
        customer = Customer(name)   # Активизировать другие объекты
        customer.order(self.server) # Клиент делает заказ официанту
        self.chef.work()
        self.oven.bake()
        customer.pay(self.server)

        
if __name__ == '__main__':
    scene = PizzaShop()             # Создать составной объект
    scene.order('Homer')            # Имитировать заказ клиента Homer
    print('...')
    scene.order('Shaggy')           # Имитировать заказ клиента Shaggy

Homer orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Homer pays for item to <Employee: name=Pat, salary=40000>
...
Shaggy orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Shaggy pays for item to <Employee: name=Pat, salary=40000>


Класс `PizzaShop` – это **контейнер и контроллер** – это конструктор, который создает и встраивает экземпляры классов работников, написанные нами в предыдущем разделе, а  также экземпляры класса `Oven`, который определен здесь.

### Еще раз об обработке потоков

Рассмотрим более реалистичный пример использования приема композиции

In [3]:
def processor(reader, converter, writer):
    while True:
        data = reader.read()
        if not data: break
        data = converter(data)
        writer.write(data)

Однако вместо простой функции мы могли бы реализовать обработку в  виде
класса, который использует прием композиции, чтобы обеспечить поддержку
наследования и более удобную конструкцию программного кода.

In [4]:
class Processor:
    def __init__(self, reader, writer):
        self.reader = reader
        self.writer = writer
        
    def process(self):
        while True:
            data = self.reader.readline()
            if not data: break
            data = self.converter(data)
            self.writer.write(data)
            
    def converter(self, data):
        assert False, 'converter must be defined' # Или возбудить исключение

Этот класс определяет метод `converter`, который, как ожидается будет перепопределен в подклассах. Это пример использования абстрактных суперклассов. При таком подходе объекты чтения (`reader`) и записи (`writer`) встраиваются в экземпляр класса (*композиция*), а логика преобразования поставляется в виде подкласса (*наследование*), а не в виде отдельной функции.

Ниже пример `converter`

In [5]:
class Uppercase(Processor):
    def converter(self, data):
        return data.upper()

In [6]:
import sys

obj = Uppercase(open('./exercises/6_30/small.py'), sys.stdout)
obj.process()

X = 1
Y = [1, 2]


Здесь класс `Uppercase` наследует логику цикла обработки потока данных (и все остальное, что может присутствовать в суперклассах). В нем необходимо определить лишь то, что будет уникальным для него, - логику преобразования данных.

Ниже пример, где вывод осуществляется через класс, который обертывает выводимый текст в теги HTML

In [7]:
class HTMLize:
    def write(self, line):
        print('<PRE>%s</PRE>' % line.rstrip())
        
Uppercase(open('./exercises/6_30/small.py'), HTMLize()).process()

<PRE>X = 1</PRE>
<PRE>Y = [1, 2]</PRE>


Если проследить порядок выполнения этого примера, можно заметить, что
было получено два варианта преобразований – приведение символов к верхнему регистру (наследованием) и преобразование в формат HTML (композицией), хотя основная логика обработки в оригинальном суперклассе `Processor` ничего не знает ни об одном из них.

Программному коду, выполняющему обработку, нужны только метод `write` – в классах, выполняющих запись, и метод `convert`. Его совершенно не интересует, что делают эти методы. Такой полиморфизм и инкапсуляция логики составляют основу такой мощи классов.

## ООП и делегирование: объекты-обертки

Под термином **делегирование** обычно подразумевается наличие объекта-контроллера, куда встраиваются другие объекты, получающие запросы на выполнение операций.

В Python делегирование часто реализуется с помощью метода `__getattr__`, потому что он перехватывает попытки доступа к несуществующим атрибутам.

**Класс-обертка (прокси-класс)** может использовать метод `__getattr__` для перенаправления обращений к обернутому объекту. Класс-обертка повторяет интерфейс обернутого объекта и может добавлять дополнительные операции.

In [8]:
# trace.py

class wrapper:
    def __init__(self, obj):
        self.wrapped = obj        # Сохранить объект

    def __getattr__(self, attrname):
        print('Trace:', attrname)    # Отметить факт извлечения
        return getattr(self.wrapped, attrname) # Делегировать извлечение

Метод `__getattr__` получает имя атрибута в виде строки. В этом примере для извлечения из обернутого объекта атрибута, имя которого представлено в виде строки, используется встроенная функция `getattr` –
вызов `getattr(X, N)` аналогичен выражению `X.N` за исключением того, что `N` – это выражение, которое во время выполнения представлено строкой, а не именем переменной. Фактически вызов `getattr(X, N)` по его действию можно сравнить с выражением `X.__dict__[N]`, только в первом случае дополнительно выполняется поиск в дереве наследования, как в выражении `X.N`, а во втором – нет.

In [9]:
x = wrapper([1, 2, 3])  # обернуть список
x.append(4)             # делегировать операцию методу списка

Trace: append


In [10]:
x.wrapped  # вывести обернутый объект

[1, 2, 3, 4]

In [11]:
x = wrapper({'a': 1, 'b': 2})  # обернуть словарь
x.keys()                       # делегировать операцию методу словаря

Trace: keys


dict_keys(['a', 'b'])

>В результате интерфейс обернутого объекта расширяется за счет методов класса-обертки

## Псевдочастные атрибуты класса

Python поддерживает такое понятие, как **"искажение" (mangling) имен (то есть расширение** с целью придать им черты локальных имен для класса. Искаженные имена иногда ошибочно называют "частными атрибутами", но в действительности это всего лишь способ **ограничить** доступ к именам в классе - искажение имени не предотвращает доступ из программного кода, находящегося за пределами класса.

Эта особенность в основном предназначена, чтобы избежать конфликтов имен в экземплярах, а не для ограничения доступа к именам.

Фактически псевдочастные имена не всегда используются даже тогда, когда
их следовало бы использовать. Гораздо чаще программисты дают внутренним
атрибутам имена, начинающиеся с  одного символа подчеркивания (например, `_X`), – согласно неофициальным соглашениям, атрибуты с такими именами не должны изменяться за пределами класса (для самого интерпретатора такие имена не имеют специального значения).

### Об искажении имен в общих чертах

>Имена внутри конструкции `class`, которые начинаются с двух символов подчеркивания, но не заканчиваются двумя символами подчеркивания, автоматически расширяются за счет включения имени вмещающего класса.

>Например, такое имя, как `__X`, в классе с именем `Spam` автоматически изменится на `_Spam__X`

Искажение имен происходит только внутри инструкций `class` и только для имен, которые начинаются двумя символами подчеркивания. Поскольку экземпляр может получать атрибуты более чем из одного класса, такое искажение позволяет избежать конфликтов.

### Для чего нужны псевдочастные атрибуты?

Задача, которую призваны решить псевдочастные атрибуты, состоит в  том,
чтобы обеспечить способ сохранности атрибутов экземпляра. В языке Python все атрибуты экземпляра принадлежат единственному объекту экземпляра.

Всякий раз, когда в пределах метода класса выполняется присваивание атрибуту аргумента `self` (например, `self.attr = value`), создается или изменяется атрибут экземпляра (поиск в  дереве наследования выполняется только при попытке получить ссылку, а  не присвоить значение). Это верно, даже когда несколько классов в  иерархии выполняют присваивание одному и тому же атрибуту, поэтому конфликты имен вполне возможны.

Предположим, что, когда программист писал класс, он предполагал, что экземпляры этого класса будут владеть атрибутом `X`. В методах класса
выполняется присваивание этому атрибуту и  позднее извлекается его значение:
```
class C1:
    def meth1(self): self.X = 88 # Предпол-ся, что X - это мой атрибут
    def meth2(self): print(self.X)
```
Далее предположим, что другой программист, работающий отдельно, исходил
из того же предположения, когда писал свой класс:
```
class C2:
    def metha(self): self.X = 99 # И мой тоже
    def methb(self): print(self.X)
```
Каждый класс по отдельности работает нормально. Проблема возникает, когда оба класса оказываются в одном дереве наследования:
```
class C3(C1, C2): ...
I = C3() # У меня только один атрибут X!
```
Теперь значение, которое получит каждый класс из выражения `self.X`, будет зависеть от того, кто из них последним присвоил значение.

Чтобы гарантировать принадлежность атрибута тому классу, который его использует, достаточно в начале имени атрибута поставить два символа подчеркивания везде, где оно используется классом

In [13]:
# private.py
class C1:
    def meth1(self): self.__X = 88    # Теперь X - мой атрибут
    def meth2(self): print(self.__X)  # Превратится в _C1__X

class C2:
    def metha(self): self.__X = 99    # И мой тоже
    def methb(self): print(self.__X)  # Превратится в _C2__X

class C3(C1, C2):
    pass

I = C3() # В I два имени X

In [14]:
I.meth1(); I.metha()
print(I.__dict__)

{'_C1__X': 88, '_C2__X': 99}


In [15]:
I.meth2(); I.methb()

88
99


Этот прием помогает избежать конфликтов имен в экземплярах, но заметьте, что он не обеспечивает настоящего сокрытия данных. Если вы знаете имя вмещающего класса, вы сможете обратиться к их атрибутам из любой точки программы, где имеется ссылка на экземпляр, используя для этого расширенное имя (например, `I._C1__X = 77`). С другой стороны, эта особенность делает **менее вероятными случайные конфликты с существующими именами в классе**.

```
class Super:
    def method(self): ...   # Фактический прикладной метод

class Tool:
    def __method(self): ... # Получит имя _Tool__method
    def other(self):
        self.__method()     # Используется внутренний метод

class Sub1(Tool, Super): ...
    def actions(self):
        self.method()       # Вызовет метод Super.method

class Sub2(Tool):
    def __init__(self):
        self.method = 99    # Не уничтожит метод Tool.__method
```

## Методы – это объекты: связанные и несвязанные методы

**Методы** - это разновидность объектов, напоминающая функции, - они могут присваиваться переменным, передаваться функциям, сохраняться в структурах данных и т.д.

Фактически, существует 2 разновидности методов:

* **Несвязанные методы класса: без аргумента `self`**. Попытка обращения к функциональному атрибуту класса через имя класса возвращает объект несвязанного метода. Чтобы вызвать этот метод, необходимо явно передать ему объект экземпляра в виде первого аргумента. В Python 3 несвязанные методы напоминают простые функции и могут вызываться через имя класса.


* **Связанные методы экземпляра: пара `self` + функция**. Попытка обращения к функциональному атрибуту класса через имя экземпляра возвращает объект связанного метода. Интерпретатор автоматически упаковывает экземпляр с функцией в объект связанного метода, поэтому вам не требуется передавать экземпляр в вызов такого метода.

Обе разновидности методов - это полноценные объекты.

Объекты связанных методов обычно взаимозаменяемы с объектами простых функций и создание их особенно полезно в случае интерфейсов, изначально ориентированных на использование функций.

Предположим, что имется следующее определение класса:

In [16]:
class Spam:
    def doit(self, message):
        print(message)

В обычной ситуации мы создаем экземпляр и сразу же вызываем его метод для вывода содержимого аргумента:

In [17]:
object1 = Spam()
object1.doit('hello world')

hello world


Однако в действительности попутно создается объект **связанного метода** - как раз перед круглыми скобками в вызове метода. Т.е. можно получить связанный метод и без его вызова. Квалифицированное имя `object.name` - это выражение, которое возвращает объект. Можно присвоить этот связанный метод другому имени и затем использовать это имя для вызова, как простую функцию:

In [18]:
object1 = Spam()
x = object1.doit  # Объект связанного метода: экземпляр + функция
x('hello world')  # то же, что и object1.doit('hello world')

hello world


In [19]:
type(x)  # это связанный метод

method

С другой стороны, если для получения метода `doit` использовать имя класса, мы получим объект **несвязанного метода**, который просто ссылается на объект функции. Чтобы вызвать метод этого типа, необходимо явно передавать экземпляр класса в первом аргументе:

In [20]:
object1 = Spam()
t = Spam.doit        # Объект несвязанного метода
t(object1, 'howdy')  # Передать экземпляр

howdy


In [21]:
type(t)  # в Python 3 это просто функция

function

Те же правила действуют внутри методов класса, когда используются атрибуты аргумента `self`, которые ссылаются на функции в классе. Выражение `self.method` возвращает объект связанного метода, потому что `self` - это объект экземпляра:

In [22]:
class Eggs:
    def m1(self, n):
        print(n)
    def m2(self):
        x = self.m1  # еще один объект связанного метода
        x(42)
        
Eggs().m2()

42


### В Python 3.0 несвязанные методы являются функциями

В Python 3.0 было **ликвидировано понятие несвязанных методов**. Методы, которые в этом разделе описываются как несвязанные методы, в версии 3.0 **обрабатываются как обычные функции**. В большинстве ситуаций это никак не влияет на программный код – в любом случае при вызове метода относительно экземпляра в первом аргументе ему будет передан сам экземпляр.

В версии 3.0 метод может вызываться без передачи ему ссылки на
экземпляр при условии, что сам метод не ожидает ее получить и вызывается исключительно через обращение к  имени класса. То есть в 
Python 3.0 ссылка на экземпляр передается методу, только когда он вызывается относительно экземпляра. При вызове метода через имя класса передавать ему экземпляр требуется, только если он ожидает получить его

In [23]:
class Selfless:
    def __init__(self, data):
        self.data = data
        
    def selfless(arg1, arg2):  # простая функция в 3.0
        return arg1 + arg2
    
    def normal(self, arg1, arg2):  # ожидает получить экз. при вызове
        return self.data + arg1 + arg2

In [24]:
X = Selfless(2)
X.normal(3, 4)   # экземпляр передается автоматически

9

In [25]:
Selfless.normal(X, 3, 4)  # экземпляр передается вручную

9

In [26]:
Selfless.selfless(3, 4)  # вызов без экземпляра (метод его и не ждет)

7

In [27]:
# ошибка - вызов относительно экземпляра
# (экземпляр передается автоматически как аргумент)
X.selfless(3, 4)

TypeError: selfless() takes 2 positional arguments but 3 were given

In [28]:
Selfless.normal(3, 4)  # ошибка - не получает ожидаемый экземпляр

TypeError: normal() missing 1 required positional argument: 'arg2'

### Связанные методы и другие вызываемые объекты

Связанные методы могут интерпретироваться как обычные вызываемые объекты, то есть как обычные функции, - они могут произвольно передаваться между компонентами программы.

Ниже пример сохранения четырех объектов связанных методов в списке и вызов их как обычных функций:

In [29]:
class Number:
    def __init__(self, base):
        self.base = base
        
    def double(self):
        return self.base * 2
    
    def triple(self):
        return self.base * 3

In [30]:
x = Number(2)  # Объекты экземпляров класса
y = Number(3)  # Атрибуты + методы
z = Number(4)
x.double()     # Обычный непосредственный вызов

4

In [31]:
# Список связанных методов, вызовы откладываются
acts = [x.double, y.double, y.triple, z.double]

for act in acts:
    print(act())    # Вызов как функции

4
6
9
8


Как и простые функции, объекты связанных методов обладают информацией,
позволяющей провести интроспекцию, включая атрибуты, обеспечивающие
доступ к объекту экземпляра и к методу. Вызов связанного метода просто задействует эту пару:

In [32]:
bound = x.double
bound.__self__, bound.__func__

(<__main__.Number at 0x7f892c46f4e0>, <function __main__.Number.double(self)>)

In [33]:
bound.__self__.base

2

In [34]:
bound()

4

Простые функции, определенные с  помощью инструкции `def` или `lambda`, экземпляры, наследующие метод `__call__`, и связанные методы экземпляров могут обрабатываться и вызываться одинаковыми способами:

In [35]:
def square(arg):
    return arg ** 2  # простые функции

class Sum:
    def __init__(self, val):  # вызываемые экземпляры
        self.val = val
    
    def __call__(self, arg):
        return self.val + arg
    
class Product:
    def __init__(self, val):   # связанные методы
        self.val = val
        
    def method(self, arg):
        return self.val * arg

In [36]:
sobject = Sum(2)
pobject = Product(3)

actions = [square, sobject, pobject.method]  # ф-ия, экземпляр, метод

for act in actions:   # все 3 вызываются одинаково
    print(act(5))     # вызов любого вызываемого с 1 аргументом

25
7
15


In [37]:
actions[-1](5)  # индексы, так же генераторы, отображения и пр.

15

Технически классы также принадлежат к категории вызываемых объектов, но обычно они вызываются для создания экземпляров, а не для выполнения какой-либо фактической работы, как показано ниже:

In [38]:
class Negate:
    def __init__(self, val):
        self.val = -val
        
    def __repr__(self):
        return str(self.val)

In [39]:
# вызвать класс тоже
actions = [square, sobject, pobject.method, Negate]

for act in actions:
    print(act(5))

25
7
15
-5


In [40]:
table = {act(5): act for act in actions}

for k, v in table.items():
    print(f'{k} => {v}')

25 => <function square at 0x7f892c4debf8>
7 => <__main__.Sum object at 0x7f892c47b048>
15 => <bound method Product.method of <__main__.Product object at 0x7f892c4e0c50>>
-5 => <class '__main__.Negate'>


## Множественное наследование: примесные классы

**Множественное наследование** - класс и его экземпляры наследуют имена из всех перечисленных суперклассов.

Поиск атрибутов производится по уровням дерева наследования - слева направо и снизу вверх.

Самый распространенный случай, когда используется множественное наследование - это смешивание методов общего назначения из нескольких суперклассов. Обычно такие суперклассы называются **примесными** - они предоставляют методы, которые добавляются в прикладные классы наследованием.

### Создание примесных классов, реализующих вывод

Определив методы вывода в суперклассе один раз, мы сможем повторно использовать его везде, где потребуется задейстовать форматированный вывод, пример класс `AttrDisplay` в главе 27.

#### Получение списка атрибутов экземпляра с помощью `__dict__`

Пример примесного класса `ListInstance`, реализующего метод `__str__`

In [41]:
# Файл lister.py
class ListInstance:
    """
    Примесный класс, реализующий получение форматированной строки при вызове
    функций print() и str() с экземпляром в виде аргумента, через наследование
    метода __str__, реализованного здесь; отображает только атрибуты
    экземпляра; self – экземпляр самого нижнего класса в дереве наследования;
    во избежание конфликтов с именами атрибутов клиентских классов использует
    имена вида __X
    """
    def __str__(self):
        return '<Instance of %s, address %s:\n%s>' % (
            self.__class__.__name__,  # Имя клиентского класса
            id(self),                 # Адрес экземпляра
            self.__attrnames())       # Список пар name=value

    def __attrnames(self):
        result = ''
        for attr in sorted(self.__dict__):  # Словарь атрибутов
            result += '\tname %s=%s\n' % (attr, self.__dict__ [attr])
        return result

In [42]:
class Spam(ListInstance):  # наследует метод __str__
    def __init__(self):
        self.data1 = 'food'
        
x = Spam()
print(x)

<Instance of Spam, address 140227131096704:
	name data1=food
>


In [43]:
x  # по умолчанию используется __repr__

Класс `ListInstance` пригодится в любых классах, даже в имеющих один и более суперклассов.

In [44]:
# Файл testmixin.py
# from lister import *  # Импортировать инструментальные классы
class Super:
    def __init__(self):          # Метод __init__ суперкласса
        self.data1 = 'spam'      # Создать атрибуты экземпляра
    def ham(self):
        pass

class Sub(Super, ListInstance):  # Подмешать методы ham и __str__
    def __init__(self):          # Инструм-ые классы имеют доступ к self
        Super.__init__(self)
        self.data2 = 'eggs'      # Добавить атрибуты экземпляра
        self.data3 = 42
    def spam(self):              # Определить еще один метод
        pass


if __name__ == '__main__':
    X = Sub()
    print(X)                     # Вызовет подмешанный метод __str__

<Instance of Sub, address 140227130145032:
	name data1=spam
	name data2=eggs
	name data3=42
>


### Получение списка атрибутов экземпляра с помощью функции `dir`

В настоящий момент `ListInstance` отображает только атрибуты экземпляра (т.е. имена, присоединенные к самому объекту экземпляра).

Но несложно сделать так, чтобы отражались и имена, унаследованные от его классов - нужно вместо сканирования `__dict__` использовать встроенную функцию `dir`, которая возвращает список всех унаследованных атрибутов.

In [45]:
# Файл lister.py, продолжение
class ListInherited:
    """
    Использует функцию dir() для получения списка атрибутов самого экземпляра
    и атрибутов, унаследованных экземпляром от его классов; в Python 3.0
    выводится больше имен атрибутов, чем в 2.6, потому что классы нового стиля
    в конечном итоге наследуют суперкласс object; метод getattr() позволяет
    получить значения унаследованных атрибутов, отсутствующих в self.__dict__;
    реализует метод __str__, а не __repr__, потому что в противном случае
    данная реализация может попасть в бесконечный цикл при выводе связанных
    методов!
    """
    def __str__(self):
        return '<Instance of %s, address %s:\n%s>' % (
            self.__class__.__name__,  # Имя класса экземпляра
            id(self),                 # Адрес экземпляра
            self.__attrnames())       # Список пар name=value
            
    def __attrnames(self):
        result = ''
        for attr in dir(self):   # Передать экземпляр функции dir()
            if attr[:2] == '__' and attr[-2:] == '__': # Пропустить
                result += '\tname %s=<>\n' % attr      # внутренние имена
            else:
                result += '\tname %s=%s\n' % (attr, getattr(self, attr))
        return result

Здесь также используется встроенная функция `getattr` для извлечения значений атрибутов по именам в виде строк. Функция `getattr` поддерживает поиск имен в дереве наследования (некоторые имена, доступные экземпляру, не принадлежат самому экземпляру).

In [46]:
# Файл testmixin.py
# from lister import *  # Импортировать инструментальные классы
class Super:
    def __init__(self):          # Метод __init__ суперкласса
        self.data1 = 'spam'      # Создать атрибуты экземпляра
    def ham(self):
        pass

class Sub(Super, ListInherited):  # Подмешать методы ham и __str__
    def __init__(self):          # Инструм-ые классы имеют доступ к self
        Super.__init__(self)
        self.data2 = 'eggs'      # Добавить атрибуты экземпляра
        self.data3 = 42
    def spam(self):              # Определить еще один метод
        pass


if __name__ == '__main__':
    X = Sub()
    print(X)                     # Вызовет подмешанный метод __str__

<Instance of Sub, address 140227130145984:
	name _ListInherited__attrnames=<bound method ListInherited.__attrnames of <__main__.Sub object at 0x7f892c47bcc0>>
	name __class__=<>
	name __delattr__=<>
	name __dict__=<>
	name __dir__=<>
	name __doc__=<>
	name __eq__=<>
	name __format__=<>
	name __ge__=<>
	name __getattribute__=<>
	name __gt__=<>
	name __hash__=<>
	name __init__=<>
	name __init_subclass__=<>
	name __le__=<>
	name __lt__=<>
	name __module__=<>
	name __ne__=<>
	name __new__=<>
	name __reduce__=<>
	name __reduce_ex__=<>
	name __repr__=<>
	name __setattr__=<>
	name __sizeof__=<>
	name __str__=<>
	name __subclasshook__=<>
	name __weakref__=<>
	name data1=spam
	name data2=eggs
	name data3=42
	name ham=<bound method Super.ham of <__main__.Sub object at 0x7f892c47bcc0>>
	name spam=<bound method Sub.spam of <__main__.Sub object at 0x7f892c47bcc0>>
>


>Следует также заметить, что теперь, когда мы предусматриваем вывод унаследованных методов, мы должны поместить реализацию перегрузки вывода в метод `__str__`, а не в `__repr__`. В методе `__repr__` эта реализация будет попадать в бесконечный цикл – при попытке отобразить значение метода вызывается метод `__repr__` класса, которому принадлежит отображаемый метод, чтобы вывести информацию о классе. То есть, если метод `__repr__` класса `ListInherited` попытается отобразить метод, тогда при выводе информации о классе, которому принадлежит отображаемый метод, снова будет вызван метод `__repr__` класса `ListInherited`. Эту проблему сложно заметить, но она существует! Измените имя метода `__str__` на `__repr__`, чтобы убедиться в этом. Если вам необходимо
использовать метод `__repr__` в  подобной ситуации, вы можете избежать зацикливания, сравнивая тип атрибута со значением `types.MethodType` из стандартной библиотеки с помощью функции `isinstance` и пропуская элементы, для которых функция вернет значение `True`.

### Получение списка атрибутов с привязкой к объектам в дереве классов

В данный момент класс `ListInherited` ничего не сообщает о том, из каких классов были унаследованы те или иные имена. Следующий примесный класс выводит схему всего дерева классов, попутно отображая атрибуты, присоединенные к каждому объекту. Это достигается за счет обхода дерева наследования, от атрибута `__class__` экземпляра к его классу и затем рекурсивно от атрибута `__bases__` класса ко всем суперклассам, сканируя словари `__dict__` в процессе обхода.

In [47]:
# Файл lister.py, продолжение
class ListTree:
    """
    Примесный класс, в котором метод __str__ просматривает все дерево классов
    и составляет список атрибутов всех объектов, находящихся в дереве выше
    self; вызывается функциями print(), str() и возвращает сконструированную
    строку со списком; во избежание конфликтов с именами атрибутов клиентских
    классов использует имена вида __X; для рекурсивного обхода суперклассов
    использует выражение-генератор; чтобы сделать подстановку значений более
    очевидной, использует метод str.format()
    """
    def __str__(self):
        self.__visited = {}
        return '<Instance of {0}, address {1}:\n{2}{3}>'.format(
            self.__class__.__name__,
            id(self),
            self.__attrnames(self, 0),
            self.__listclass(self.__class__, 4))
    
    def __listclass(self, aClass, indent):
        dots = '.' * indent
        if aClass in self.__visited:
            return '\n{0}<Class {1}:, address {2}: (see above)>\n'.format(
                            dots,
                            aClass.__name__,
                            id(aClass))
        else:
            self.__visited[aClass] = True
            genabove = (self.__listclass(c, indent+4)
                                for c in aClass.__bases__)
            return '\n{0}<Class {1}, address {2}:\n{3}{4}{5}>\n'.format(
                                dots,
                                aClass.__name__,
                                id(aClass),
                                self.__attrnames(aClass, indent),
                                ''.join(genabove),
                                dots)
    
    def __attrnames(self, obj, indent):
        spaces = ' ' * (indent + 4)
        result = ''
        for attr in sorted(obj.__dict__):
            if attr.startswith('__') and attr.endswith('__'):
                result += spaces + '{0}=<>\n'.format(attr)
            else:
                result += spaces + '{0}={1}\n'.format(attr, getattr(obj,
                                                        attr))
        return result

In [48]:
# Файл testmixin.py
# from lister import *  # Импортировать инструментальные классы
class Super:
    def __init__(self):          # Метод __init__ суперкласса
        self.data1 = 'spam'      # Создать атрибуты экземпляра
    def ham(self):
        pass

class Sub(Super, ListTree):  # Подмешать методы ham и __str__
    def __init__(self):          # Инструм-ые классы имеют доступ к self
        Super.__init__(self)
        self.data2 = 'eggs'      # Добавить атрибуты экземпляра
        self.data3 = 42
    def spam(self):              # Определить еще один метод
        pass


if __name__ == '__main__':
    X = Sub()
    print(X)                     # Вызовет подмешанный метод __str__

<Instance of Sub, address 140227130556600:
    _ListTree__visited={}
    data1=spam
    data2=eggs
    data3=42

....<Class Sub, address 93948491872536:
        __doc__=<>
        __init__=<>
        __module__=<>
        spam=<function Sub.spam at 0x7f892c47f950>

........<Class Super, address 93948491873480:
            __dict__=<>
            __doc__=<>
            __init__=<>
            __module__=<>
            __weakref__=<>
            ham=<function Super.ham at 0x7f892c47f840>

............<Class object, address 93948455948448:
                __class__=<>
                __delattr__=<>
                __dir__=<>
                __doc__=<>
                __eq__=<>
                __format__=<>
                __ge__=<>
                __getattribute__=<>
                __gt__=<>
                __hash__=<>
                __init__=<>
                __init_subclass__=<>
                __le__=<>
                __lt__=<>
                __ne__=<>
                __new__=<>
 

**Поддержка слотов**: поскольку классы `ListInstance` и `ListTree`,
представленные здесь, выполняют сканирование словарей экземпляров, они не поддерживают атрибуты, хранящиеся в **слотах** – новой и  относительно редко используемой особенности, с  которой мы встретимся в  следующей главе, где мы увидим, как атрибуты экземпляров объявляются в  атрибуте `__slots__` класса).

## Классы – это объекты: универсальные фабрики объектов

Иногда бывает необходимо, чтобы объекты создавались в ответ на сложившиеся условия, которые невозможно предсказать на этапе разработки программы.

**Фабричный шаблон проектирования** позволяет реализовать такой подход.
Можно передавать классы функциям, которые создают объекты произвольных типов, – в  кругах, связанных с  ООП, такие функции иногда называют фабриками. Синтаксическая конструкция, с  которой мы познакомились в  главе  18, может вызывать любые классы с любым числом аргументов конструкторов за один присест, генерируя экземпляр любого типа:

In [49]:
def factory(aClass, *args, **kwargs):  # Кортеж с переменным числом аргументов
    return aClass(*args, **kwargs)     # Вызов aClass


class Spam:
    def doit(self, message):
        print(message)

        
class Person:
    def __init__(self, name, job):
        self.name = name
        self.job = job
        

object1 = factory(Spam)  # Создать объект Spam
object2 = factory(Person, 'Guido', 'guru') # Создать объект Person

object2.__dict__

{'name': 'Guido', 'job': 'guru'}

В этом фрагменте определена **функция-генератор объектов с  именем `factory`**. Она ожидает получить объект класса (любого) вместе с одним или более аргументами конструктора класса. Функция использует специальный синтаксис вызова с переменным числом аргументов, чтобы создать и вернуть экземпляр.

### Зачем нужны фабрики?

Фабрика могла бы помочь изолировать программный код от динамически настраиваемой конструкции объекта.